# imports

In [2]:
import pandas as pd

## constants

In [3]:
NUMBER_OF_ELEMENTS_TO_TAKE = 200

# Preprocess dataframe

## Load data and concat two years

In [4]:
data_2018 = pd.read_csv("data2018.csv", encoding="cp1251")
data_2019 = pd.read_csv("data2019.csv", encoding="cp1251")
df = pd.concat([data_2018, data_2019])
df = df.sample(1100000, random_state=2).reset_index(drop=True)
df

,Рік,Область,Учбовий заклад,Спеціальність,Факультет,Місце у рейтингу,Пріоритет,Конкурсний бал,Статус заяви,Інфо про спеціальність,Квоти
0,2019,м. Київ,Київський національний торговельно-економічний...,081 Право,Факультет міжнародної торгівлі та права,754,К,155.900,Допущено,Українська мова та література (ЗНО): 154\r\nІс...,—
1,2018,Львівська область,Львівський національний університет імені Іван...,035 Філологія,Філологічний факультет,390,К,165.852,Допущено,Середній бал документа про освіту: 10.6\r\nУкр...,—
2,2019,Львівська область,Львівський національний університет імені Іван...,"291 Міжнародні відносини, суспільні комунікаці...",Факультет міжнародних відносин,480,4,187.068,Допущено,—,—
3,2018,м. Київ,Київський національний лінгвістичний університет,081 Право,Факультет економіки і права,26,6,193.850,Рекомендовано (контракт),Українська мова та література (ЗНО): 193\r\nІс...,—
4,2019,м. Київ,Національний університет харчових технологій,073 Менеджмент,Навчально-науковий інститут економіки і управл...,162,К,119.400,Рекомендовано (к),Українська мова та література (ЗНО): 131\r\nМа...,—
...,...,...,...,...,...,...,...,...,...,...,...
1099995,2018,Львівська область,Львівський національний університет імені Іван...,081 Право,Юридичний факультет,606,5,192.780,Допущено,Українська мова та література (ЗНО): 191\r\nІс...,—
1099996,2018,Львівська область,Львівський національний університет імені Іван...,061 Журналістика,Факультет журналістики,949,4,169.065,Допущено,Українська мова та література (ЗНО): 173\r\nІс...,—
1099997,2019,Івано-Франківська область,Прикарпатський національний університет імені ...,241 Готельно-ресторанна справа,Факультет туризму,89,6,159.171,Допущено,Українська мова та література (ЗНО): 164\r\nІн...,—
1099998,2018,м. Київ,Державний вищий навчальний заклад «Київський н...,051 Економіка,Факультет мiжнародної економiки i менеджменту,243,2,195.200,Рекомендовано (контракт),Українська мова та література (ЗНО): 196\r\nМа...,—


## Rename, remove and clean columns
0 - didn't pass at all

1 - pass to contract

2 - pass to budget

In [5]:
df.columns = [
    "year",
    "district",
    "university",
    "specialty",
    "faculty",
    "rating",
    "priority",
    "points",
    "status",
    "info",
    "quotas"
]
df = df.drop(["year", "district", "faculty", "info", "quotas", "rating", "priority"], axis=1)
df["status"] = df["status"].map({
    'До наказу (контракт)': 1,
    'Рекомендовано (контракт)': 1,
    'Відмова': 0,
    'Скасовано НЗ': 0,
    'Скасовано (втрата пріор.)': 0,
    'Зареєстровано': 0,
    'Затримано': 0,
    'Відраховано': 0,
    'До наказу (бюджет)': 2,
    'Відхилено (бюджет)': 0,
    'Допущено': 0,
    'Рекомендовано (бюджет)': 2,
    'Заява надійшла з сайту': 0,
    'Відхилено (контракт)': 0,
    'Допущено (контракт, за ріш. ПК)': 1,
    'Рекомендовано (к)': 1,
    'До наказу (к)': 1,
    'До наказу (б)': 2,
    'Відхилено (б)': 0,
    'Рекомендовано (б)': 2,
    'Відхилено (к)': 0,
    'Допущено (к, за ріш. ПК)': 1
})
df["specialty"] = df["specialty"].map(lambda val: val[:3])

In [7]:
df = df.query("points <= 200 and points >= 100")
specialties = df.specialty.unique()
universities = df.university.unique()
accum = []
for spec in specialties:
    accum.append(df.loc[(df["specialty"] == spec) & (df["status"] == 0)][:NUMBER_OF_ELEMENTS_TO_TAKE])
    accum.append(df.loc[(df["specialty"] == spec) & (df["status"] == 1)][:NUMBER_OF_ELEMENTS_TO_TAKE])
    accum.append(df.loc[(df["specialty"] == spec) & (df["status"] == 2)][:NUMBER_OF_ELEMENTS_TO_TAKE])
for un in universities:
    accum.append(df.loc[(df["university"] == un) & (df["status"] == 0)][:NUMBER_OF_ELEMENTS_TO_TAKE])
    accum.append(df.loc[(df["university"] == un) & (df["status"] == 1)][:NUMBER_OF_ELEMENTS_TO_TAKE])
    accum.append(df.loc[(df["university"] == un) & (df["status"] == 2)][:NUMBER_OF_ELEMENTS_TO_TAKE])
df = pd.concat(accum)
df

,university,specialty,points,status
0,Київський національний торговельно-економічний...,081,155.900,0
38,Херсонський державний університет,081,131.144,0
60,Хмельницький університет управління та права,081,121.888,0
95,Київський національний університет культури і ...,081,145.500,0
123,Харківський національний університет внутрішні...,081,126.429,0
...,...,...,...,...
1039510,Уманська філія Приватного вищого навчального з...,073,140.454,1
875320,Кам`янський медичний коледж,223,183.000,0
907039,Надвірнянський коледж Національного транспортн...,029,127.000,0
919912,Вугледарський коледж Донецького державного уні...,072,132.000,0


In [8]:
budget = df.query("status == 2")
contract = df.query("status == 1").sample(n=5000)
not_passed = df.query("status == 0").sample(n=2000)
df = pd.concat([budget, contract, not_passed])
df

,university,specialty,points,status
108,Львівський національний університет імені Іван...,081,189.210,2
861,Університет державної фіскальної служби України,081,134.028,2
2299,Національний юридичний університет імені Яросл...,081,187.476,2
2779,Університет державної фіскальної служби України,081,178.959,2
2829,Національний юридичний університет імені Яросл...,081,186.762,2
...,...,...,...,...
1437,Київський національний університет імені Тарас...,052,172.900,0
941523,Білоцерківський національний аграрний університет,201,143.991,0
91969,Державний вищий навчальний заклад «Український...,181,113.067,0
330264,Придніпровська державна академія фізичної куль...,017,170.719,0


In [9]:
df.status = df.status.dropna()
df.head()
df

,university,specialty,points,status
108,Львівський національний університет імені Іван...,081,189.210,2
861,Університет державної фіскальної служби України,081,134.028,2
2299,Національний юридичний університет імені Яросл...,081,187.476,2
2779,Університет державної фіскальної служби України,081,178.959,2
2829,Національний юридичний університет імені Яросл...,081,186.762,2
...,...,...,...,...
1437,Київський національний університет імені Тарас...,052,172.900,0
941523,Білоцерківський національний аграрний університет,201,143.991,0
91969,Державний вищий навчальний заклад «Український...,181,113.067,0
330264,Придніпровська державна академія фізичної куль...,017,170.719,0


## Get values for frontend

In [10]:
df.specialty.unique()

array(['081', '035', '291', '073', '072', '014', '034', '076', '051',
       '113', '027', '071', '101', '281', '201', '012', '141', '231',
       '181', '207', '162', '053', '122', '242', '016', '192', '275',
       '091', '123', '061', '017', '208', '262', '075', '106', '022',
       '241', '205', '193', '191', '292', '033', '232', '171', '226',
       '153', '121', '125', '142', '131', '227', '013', '126', '172',
       '032', '184', '224', '272', '161', '273', '133', '223', '187',
       '151', '105', '186', '023', '054', '135', '163', '052', '026',
       '274', '271', '293', '124', '031', '136', '021', '152', '025',
       '029', '202', '112', '263', '015', '103', '134', '204', '028',
       '102', '173', '194', '261', '183', '104', '111', '206', '144',
       '203', '024', '185', '229', '132', '182', '145', '143', '211',
       '011', '041', '256', '251', '255'], dtype=object)

In [11]:
df.university.unique()

array(['Львівський національний університет імені Івана Франка',
       'Університет державної фіскальної служби України',
       'Національний юридичний університет імені Ярослава Мудрого',
       'Національний університет «Львівська політехніка»',
       'Донецький національний університет імені Василя Стуса',
       'Національний університет Одеська юридична академія',
       'Східноєвропейський національний університет імені Лесі Українки',
       'Національний авіаційний університет',
       'Київський національний університет імені Тараса Шевченка',
       'Національний університет «Острозька академія»',
       'Чернігівський національний педагогічний університет імені\xa0Т.Г.\xa0Шевченка',
       'Національний університет «Києво-Могилянська академія»',
       'Сумський державний університет',
       'Національний університет кораблебудування імені адмірала Макарова',
       'Національний університет Острозька академія',
       'Національний університет «Одеська юридична академія

## Split into two datasets

- The first defines people who didn't pass to contract and didn't pass at all as the same people. This model is used to check people who wants to apply for budget
- The second defines people who pass to contract and to budget as the same category, since if you pass to budget you'll pass to contract

In [12]:
df_budget = df.copy()
df_contract = df.copy()
df_budget["status"] = df_budget["status"].map({
    1: -1,
    2: 1,
    0: -1
})
df_contract["status"] = df_contract["status"].map({
    1: 1,
    2: 1,
    0: -1
})


In [13]:
df_budget.head()

,university,specialty,points,status
108,Львівський національний університет імені Іван...,081,189.210,1
861,Університет державної фіскальної служби України,081,134.028,1
2299,Національний юридичний університет імені Яросл...,081,187.476,1
2779,Університет державної фіскальної служби України,081,178.959,1
2829,Національний юридичний університет імені Яросл...,081,186.762,1


In [14]:
df_contract.head()

,university,specialty,points,status
108,Львівський національний університет імені Іван...,081,189.210,1
861,Університет державної фіскальної служби України,081,134.028,1
2299,Національний юридичний університет імені Яросл...,081,187.476,1
2779,Університет державної фіскальної служби України,081,178.959,1
2829,Національний юридичний університет імені Яросл...,081,186.762,1


## Dump cleaned_data set

In [15]:
df_budget.to_csv("budget.csv", index=False)
df_contract.to_csv("contract.csv", index=False)